In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.gridspec as gridspec
from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import matplotlib.style as style
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import missingno as msno

############################

import warnings


from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error

warnings.simplefilter(action="ignore", category=FutureWarning)

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/energydata_complete.csv")
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

###### From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the Root Mean Squared error in three D.P?

In [ ]:
target = ["T6"]
features = ["T2"]
#y_train = df[target]
#X_train = df[features]
X = df[features]
Y = df[target]

In [ ]:
## Train test s
from sklearn.model_selection import train_test_split
## Train test split follows this distinguished code pattern and helps creating train and test set to build machine learning.
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = .33, random_state = 0)
X.shape

(19735, 1)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((13222, 1), (13222, 1), (6513, 1), (6513, 1))

In [ ]:
## importing necessary models.
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

## Call in the LinearRegression object
lin_reg = LinearRegression(n_jobs=-1)
## fit train and test data.
lin_reg.fit(X_train, y_train)
## Predict test data.
y_pred = lin_reg.predict(X_test)

In [ ]:
## get average squared error(MSE) by comparing predicted values with real values.
print ('%.2f'%mean_squared_error(y_test, y_pred))

12.95


In [ ]:
np.sqrt(12.95)

3.598610843089316

In [ ]:
## Alternatively

y_pred = np.array(y_pred)
y_test = np.array(y_test)
np.sqrt(sum((y_pred-y_test)**2)/len(y_test))

array([3.59808879])


###### Remove the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a  random state of 42 (for reproducibility). Normalize the dataset using the MinMaxScaler (Hint: Use the MinMaxScaler fit_transform and transform methods on the train and test set respectively). Run a multiple linear regression using the training set. Answer the following questions:

###### What is the Mean Absolute Error (in three decimal places) for the  training set?

In [ ]:
target = ['Appliances']
features = ['T1', 'RH_1', 'T2', 'RH_2', 'T3', 'RH_3', 'T4', 'RH_4',
       'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8', 'RH_8', 'T9', 'RH_9',
       'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility',
       'Tdewpoint', 'rv1', 'rv2']
#y_train = df[target]
#X_train = df[features]
X = df[features]
Y = df[target]

In [ ]:
df_copy = df
df_copy.drop(columns = ['date', 'lights'], inplace = True)
df_copy.info()

In [ ]:
## Train test s
from sklearn.model_selection import train_test_split
## Train test split follows this distinguished code pattern and helps creating train and test set to build machine learning.
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = .33, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((13222, 26), (6513, 26), (13222, 1), (6513, 1))

In [ ]:
# import
from sklearn.preprocessing import MinMaxScaler

# Create a MinMaxScaler instance
scaler = MinMaxScaler()

In [ ]:
data = [X_train, X_test, y_train, y_test]

# Fit and transform the data
for i in data:
  scaler.fit_transform(i)

In [79]:
## Call in the LinearRegression object
model = LinearRegression(n_jobs=-1)
## fit train and test data.
model.fit(X_train, y_train)
## Predict test data.
y_pred1 = model.predict(X_train)
y_pred2 = model.predict(X_test)

In [86]:
print("Training MAE:", round(mean_absolute_error(y_train, y_pred1), 3))

Training MAE: 53.496


###### What is the Root Mean Squared Error (in three decimal places) for the training set?

In [89]:
## get average squared error(MSE) by comparing predicted values with real values.
Training_MSE = mean_squared_error(y_train, y_pred1)

In [91]:
print("Training MSE:",round(np.sqrt(Training_MSE),3))

Training MSE: 94.839


###### What is the Mean Absolute Error (in three decimal places) for test set?

In [85]:
print("Test MAE:", round(mean_absolute_error(y_test, y_pred2), 3))

Test MAE: 53.743


###### What is the Root Mean Squared Error (in three decimal places) for test set?

In [92]:
Training_MSE_test = mean_squared_error(y_test, y_pred2)

In [93]:
print("Training MSE test:",round(np.sqrt(Training_MSE_test),3))

Training MSE test: 94.599


###### Train a lasso regression model with default value and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [96]:
from sklearn.linear_model import Lasso

# Create Lasso model
alpha = 0.01
lasso_model = Lasso(alpha=alpha)

# Fit the model
lasso_model.fit(X_train, y_train)

# Obtain feature weights (coefficients)
feature_weights = lasso_model.coef_
print("Feature Weights:", feature_weights)
print("Number non-zero feature weights:", len(feature_weights))

Feature Weights: [ -0.67651079  16.21709705 -17.68233508 -13.56123719  26.24663003
   4.82085033   2.62606177   0.86776105  -2.04489567   0.28918308
   7.19898779   0.41969253   1.14582472  -1.54250332   9.67855741
  -5.78002013 -20.8818604   -1.87549339 -10.33981945   0.19132241
  -0.94859071   2.35732979   0.20816583   5.12066278   0.0553453
   0.        ]
Number non-zero feature weights: 26


###### What is the new RMSE with the Lasso Regression on the test set?

In [99]:
# Create Lasso model
alpha = 0.01
lasso_model = Lasso(alpha=alpha)

# Fit the model
lasso_model.fit(X_train, y_train)

# Make predictions
y_pred3 = lasso_model.predict(X_test)

# Calculate RMSE
mse = mean_squared_error(y_test, y_pred3)
rmse = np.sqrt(mse)
print("RMSE with the Lasso Regression:", round(rmse, 3))

RMSE with the Lasso Regression: 94.6
